In [6]:
# !pip install opencv-python
# !pip install matplotlib
# !pip install tensorflow
# !pip install tensorflow-gpu
# !pip install --user mediapipe

  Obtaining dependency information for mediapipe from https://files.pythonhosted.org/packages/f6/10/475247e1773227f565f109110af069ee1a37b8c194ba677ea337ec988a5a/mediapipe-0.10.5-cp311-cp311-win_amd64.whl.metadata
  Using cached mediapipe-0.10.5-cp311-cp311-win_amd64.whl.metadata (9.8 kB)
  Obtaining dependency information for opencv-contrib-python from https://files.pythonhosted.org/packages/81/3c/bbb3ceee9fbefc505f98c24dafda68c7b3c4f83b6951c0712b4623fe4cce/opencv_contrib_python-4.8.1.78-cp37-abi3-win_amd64.whl.metadata
  Using cached opencv_contrib_python-4.8.1.78-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached sounddevice-0.4.6-py3-none-win_amd64.whl (199 kB)
Using cached mediapipe-0.10.5-cp311-cp311-win_amd64.whl (50.3 MB)
Using cached opencv_contrib_python-4.8.1.78-cp37-abi3-win_amd64.whl (44.8 MB)


In [1]:
import cv2
import numpy as np
import os
import time
import mediapipe as mp
from matplotlib import pyplot as plt

In [2]:
mp_holistic = mp.solutions.holistic #Holistic model
mp_drawing = mp.solutions.drawing_utils #drawing utilities

In [3]:
def mediapipe_detection(image,model):
    image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)  #color conversion
    image.flags.writeable = False                   #Image no longer writable
    results = model.process(image)                 #make predictions
    image.flags.writeable = True                   #image is now writeable
    image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR) #color conversion
    return image,results                          #return results
    

In [4]:
def draw_landmarks(image,result):
    mp_drawing.draw_landmarks(image,result.face_landmarks,mp_holistic.FACEMESH_CONTOURS)
    mp_drawing.draw_landmarks(image,result.pose_landmarks,mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image,result.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image,result.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS)

In [5]:
def draw_styled_landmarks(image,result):
    mp_drawing.draw_landmarks(image,result.face_landmarks,mp_holistic.FACEMESH_CONTOURS,
                             mp_drawing.DrawingSpec(color=(80,110,10),thickness=1,circle_radius=1),
                             mp_drawing.DrawingSpec(color=(80,256,121),thickness=1,circle_radius=1))
    mp_drawing.draw_landmarks(image,result.pose_landmarks,mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10),thickness=2,circle_radius=4),
                             mp_drawing.DrawingSpec(color=(80,44,121),thickness=2,circle_radius=2))
    mp_drawing.draw_landmarks(image,result.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(121,22,76),thickness=2,circle_radius=4),
                             mp_drawing.DrawingSpec(color=(121,44,250),thickness=2,circle_radius=2))
    mp_drawing.draw_landmarks(image,result.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=4),
                             mp_drawing.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2))

In [83]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        
        ret , frame = cap.read()  #read feed
        
        #make detection
        image,results =  mediapipe_detection(frame,holistic)
        
        #draw landmarks
        draw_styled_landmarks(image,results)
        
        # print(results)
        cv2.imshow("OpenCV Feed",image)  #show screen
        
        if cv2.waitKey(10) & 0xFF == ord('q'):  #break loop
            break
    cap.release()
    cv2.destroyAllWindows()

In [6]:
#Extracting key points 

def extract_keypoints(results):
    #pose
    pose = np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)

    #face
    face = np.array([[res.x,res.y,res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)


    #left hand
    lh = np.array([[res.x,res.y,res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)


    #right hand
    rh = np.array([[res.x,res.y,res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose,face,lh,rh])



In [7]:
DATA_PATH = os.path.join('MP_Data')

actions = np.array(["hello","thanks","iloveyou"])

no_sequences=30

sequences_length=30

In [20]:
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH,action,str(sequence)))
        except:
            pass

In [26]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
    for action in actions:
        for sequence in range(no_sequences):
            for frame_num in range(sequences_length):
                ret , frame = cap.read()  #read feed

                #make detection
                image,results =  mediapipe_detection(frame,holistic)

                #draw landmarks
                draw_styled_landmarks(image,results)
              
                # print(results)
                if frame_num == 0:
                    cv2.putText(image,'Starting Collectiob',(120,200),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),4,cv2.LINE_AA)
                    cv2.putText(image,'Collecting frames for {} Video Number {}'.format(action,sequence),(15,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),1,cv2.LINE_AA)
                    cv2.waitKey(1000)
                else:
                    cv2.putText(image,'Collecting frames for {} Video Number {}'.format(action,sequence),(15,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),1,cv2.LINE_AA)
                
                keypoints=extract_keypoints(results)
                npy_path=os.path.join(DATA_PATH,action,str(sequence),str(frame_num))
                np.save(npy_path,keypoints)
                cv2.imshow("OpenCV Feed",image)  #show screen

                if cv2.waitKey(10) & 0xFF == ord('q'):  #break loop
                    break
    cap.release()
    cv2.destroyAllWindows()

In [8]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [9]:
label_map = {label:num for num,label in enumerate(actions)}
label_map

{'hello': 0, 'thanks': 1, 'iloveyou': 2}

In [10]:
sequences = []
labels = []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequences_length):
            res = np.load(os.path.join(DATA_PATH,action,str(sequence),"{}.npy".format(str(frame_num))))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [11]:
X = np.array(sequences)
Y = to_categorical(labels).astype(int)

In [12]:
X_train , X_test, Y_train , Y_test = train_test_split(X,Y,train_size=0.05)

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense
from tensorflow.keras.callbacks import TensorBoard

In [14]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [18]:
model =  Sequential()
model.add(LSTM(64,activation='relu',return_sequences=True,input_shape=(30,1662)))
model.add(LSTM(128,activation='relu',return_sequences=True))
model.add(LSTM(64,activation='relu',return_sequences=False))
model.add(Dense(64,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(actions.shape[0],activation='softmax'))


In [25]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['categorical_accuracy'])

In [26]:
model.fit(X_train,Y_train,epochs=2000,callbacks=[tb_callback])

Epoch 1/2000
1/1 [==============================] - 10s 10s/step - loss: 1.0763 - categorical_accuracy: 0.0000e+00
Epoch 2/2000
1/1 [==============================] - 0s 95ms/step - loss: 5.7857 - categorical_accuracy: 0.5000
Epoch 3/2000
1/1 [==============================] - 0s 86ms/step - loss: 73.1823 - categorical_accuracy: 0.2500
Epoch 4/2000
1/1 [==============================] - 0s 90ms/step - loss: 104.1575 - categorical_accuracy: 0.2500
Epoch 5/2000
1/1 [==============================] - 0s 86ms/step - loss: 16.3785 - categorical_accuracy: 0.2500
Epoch 6/2000
1/1 [==============================] - 0s 90ms/step - loss: 30.0762 - categorical_accuracy: 0.2500
Epoch 7/2000
1/1 [==============================] - 0s 88ms/step - loss: 33.6280 - categorical_accuracy: 0.2500
Epoch 8/2000
1/1 [==============================] - 0s 95ms/step - loss: 6.8041 - categorical_accuracy: 0.5000
Epoch 9/2000
1/1 [==============================] - 0s 91ms/step - loss: 7.8207 - categorical_accuracy

In [27]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_9 (LSTM)               (None, 30, 64)            442112    
                                                                 
 lstm_10 (LSTM)              (None, 30, 128)           98816     
                                                                 
 lstm_11 (LSTM)              (None, 64)                49408     
                                                                 
 dense_9 (Dense)             (None, 64)                4160      
                                                                 
 dense_10 (Dense)            (None, 32)                2080      
                                                                 
 dense_11 (Dense)            (None, 3)                 99        
                                                                 
Total params: 596675 (2.28 MB)
Trainable params: 59667

In [42]:
res = model.predict(X_test)
actions[np.argmax(res[0])]
actions[np.argmax(Y_test[0])]

3/3 [==============================] - 0s 37ms/step


'thanks'

In [32]:
model.save('action.h5')

C:\Users\Rachna\anaconda3\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [33]:
from sklearn.metrics import multilabel_confusion_matrix , accuracy_score

In [43]:
yhat = model.predict(X_train)
ytrue = np.argmax(Y_train,axis=1).tolist()
yhat = np.argmax(yhat,axis=1).tolist()

1/1 [==============================] - 0s 38ms/step


In [45]:
multilabel_confusion_matrix(ytrue,yhat)

array([[[1, 0],
        [0, 3]],

       [[3, 0],
        [0, 1]]], dtype=int64)

In [46]:
accuracy_score(ytrue,yhat)

1.0

In [51]:
sequence = []
sentence = []
threshold = 0.2
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        
        ret , frame = cap.read()  #read feed
        
        #make detection
        image,results =  mediapipe_detection(frame,holistic)
        
        #draw landmarks
        draw_styled_landmarks(image,results)
        
        keypoints = extract_keypoints(results)
        sequence.insert(0,keypoints)
        sequence=sequence[:30]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence,axis=0))[0]
            print(actions[np.argmax(res)])
            
        if res[np.argmax(res)] > threshold :
            if len(sentence) > 0:
                if actions[np.argmax(res)] != sentence[-1]:
                    sentence.append(actions[np.argmax(res)])
            else:
                sentence.append(actions[np.argmax(res)])
        if len(sentence) > 5:
            sentence = sentence[-5:]
            
        cv2.rectangle(image ,(0,0) ,(640,40),(245,117,16) ,-1)
        cv2.putText(image,''.join(sentence),(3,30),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)

        # print(results)
        cv2.imshow("OpenCV Feed",image)  #show screen
        
        if cv2.waitKey(10) & 0xFF == ord('q'):  #break loop
            break
    cap.release()
    cv2.destroyAllWindows()

1/1 [==============================] - 0s 37ms/step
thanks
1/1 [==============================] - 0s 38ms/step
thanks
1/1 [==============================] - 0s 40ms/step
thanks
1/1 [==============================] - 0s 35ms/step
thanks
1/1 [==============================] - 0s 36ms/step
thanks
1/1 [==============================] - 0s 37ms/step
thanks
1/1 [==============================] - 0s 44ms/step
thanks
1/1 [==============================] - 0s 43ms/step
thanks
1/1 [==============================] - 0s 49ms/step
thanks
1/1 [==============================] - 0s 44ms/step
thanks
1/1 [==============================] - 0s 42ms/step
thanks
1/1 [==============================] - 0s 37ms/step
thanks
1/1 [==============================] - 0s 45ms/step
thanks
1/1 [==============================] - 0s 42ms/step
thanks
1/1 [==============================] - 0s 37ms/step
thanks
1/1 [==============================] - 0s 35ms/step
thanks
1/1 [==============================] - 0s 36ms/step
than